In [1]:
#import dependencies
from bs4 import BeautifulSoup as bs
from splinter import Browser
import os
import pandas as pd
import time
import requests
import pymongo
from pymongo import MongoClient
client = MongoClient()


In [12]:

# #pointing to the directory where chromedriver exists
executable_path = {"executable_path":"C:/Users/bryan/Documents/DUclass/chromedriver_win32/chromedriver"}
browser = Browser("chrome", **executable_path, headless = False)

Mars News

In [2]:
#visiting Nasa news page
# Dealing with the headline

def get_mars_headline():
    url = "https://mars.nasa.gov/api/v1/news_items/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
    resp = requests.get(url).json()
    first_item = resp.get('items')[0]
    return {"item_title": first_item.get('title'), 
            "item_desc": first_item.get('description')
           }

headlines_dict = get_mars_headline()

headlines_dict

{'item_title': 'After a Reset, Curiosity Is Operating Normally',
 'item_desc': 'Curiosity has returned to science operations and is once again exploring the clay unit. '}

Mars Facts

In [4]:
def get_mars_facts_table():
    tables = pd.read_html("https://space-facts.com/mars/") # list of tables, even if there's 1
    table_we_want = tables[0]
    table_we_want.columns = ['Name', "Value"]
    formatted =  table_we_want.to_html(classes=["table-bordered", "table-striped", "table-hover"])
    return {"html_table_facts": formatted}

facts_table_dict = get_mars_facts_table()
facts_table_dict

{'html_table_facts': '<table border="1" class="dataframe table-bordered table-striped table-hover">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Name</th>\n      <th>Value</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>0</th>\n      <td>Equatorial Diameter:</td>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>1</th>\n      <td>Polar Diameter:</td>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.42 x 10^23 kg (10.7% Earth)</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Orbit Distance:</td>\n      <td>227,943,824 km (1.52 AU)</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Orbit Period:</td>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Surface Temperature:</td>\n      <td>-153 to 20 °C</td>\n    </tr>\n    <tr>\n      <th>7</th>\

Inserting this into mongo to make sure things work

In [6]:
# Insert into Mongo - but first merge our dictionaries

merged_dict = {**facts_table_dict, **headlines_dict}

In [7]:
db = client.test_mars
collection = db.test_collection
collection.insert_one(merged_dict)

Getting the Weather on Mars

In [33]:
def get_mars_weather():
    url_weather = "https://twitter.com/marswxreport?lang=en"
    browser.visit(url_weather)
    html_weather = browser.html
    soup = bs(html_weather, "html.parser")
    return {"weather": soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text}
  
mars_weather_dict = get_mars_weather()
        

In [34]:
mars_weather_dict


{'weather': 'InSight sol 92 (2019-03-01) low -94.4ºC (-137.9ºF) high -12.9ºC (8.8ºF)\nwinds from the SW at 4.6 m/s (10.2 mph) gusting to 10.4 m/s (23.2 mph)\npressure at 7.20 hPapic.twitter.com/zxXhRFOwTo'}

Put weather into mongoDb

In [35]:
db = client.test_mars
collection = db.test_collection
collection.insert_one(mars_weather_dict)

Featured image from JPL

In [45]:
def get_ftr_image ():
    url_image = "https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA18811_ip.jpg"
    browser.visit(url_image)
    html_image = browser.html
    soup = bs(html_image, "html.parser")
    return {"ftr_image": image}

ftr_image_dict = get_ftr_image


In [46]:
ftr_image_dict

<function __main__.get_ftr_image()>

Put image into mongodb (yes I know that I can do this all at once but I'm testing as I go

In [48]:
def get_the_image():
    images = pd.read_html("https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars")
    image_we_want = image[0]
    return image_we_want.to_html()

In [50]:
get_the_image

<function __main__.get_the_image()>

In [8]:
url_image = "https://www.jpl.nasa.gov/spaceimages/?search=&category=featured#submit"
browser.visit(url_image)

In [9]:
#Getting the base url
from urllib.parse import urlsplit
base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url_image))
print(base_url)

https://www.jpl.nasa.gov/


In [10]:
#Design an xpath selector to grab the image
xpath = "//*[@id=\"page\"]/section[3]/div/ul/li[1]/a/div/div[2]/img"

In [11]:

#Use splinter to click on the mars featured image
#to bring the full resolution image
results = browser.find_by_xpath(xpath)
img = results[0]
img.click()

In [12]:
#get image url using BeautifulSoup
html_image = browser.html
soup = bs(html_image, "html.parser")
img_url = soup.find("img", class_="fancybox-image")["src"]
full_img_url = base_url + img_url
print(full_img_url)


TypeError: 'NoneType' object is not subscriptable

Mars Hemispheres

In [ ]:
url_hemisphere = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
browser.visit(url_hemisphere)

In [ ]:
#Getting the base url
hemisphere_base_url = "{0.scheme}://{0.netloc}/".format(urlsplit(url_hemisphere))
print(hemisphere_base_url)

In [ ]:
hemisphere_img_urls = []
results = browser.find_by_xpath( "//*[@id='product-section']/div[2]/div[1]/a/img").click()
time.sleep(2)
cerberus_open_click = browser.find_by_xpath( "//*[@id='wide-image-toggle']").click()
time.sleep(1)
cerberus_image = browser.html
soup = bs(cerberus_image, "html.parser")
cerberus_url = soup.find("img", class_="wide-image")["src"]
cerberus_img_url = hemisphere_base_url + cerberus_url
print(cerberus_img_url)
cerberus_title = soup.find("h2",class_="title").text
print(cerberus_title)
back_button = browser.find_by_xpath("//*[@id='splashy']/div[1]/div[1]/div[3]/section/a").click()
cerberus = {"image title":cerberus_title, "image url": cerberus_img_url}
hemisphere_img_urls.append(cerberus)

In [ ]:
results1 = browser.find_by_xpath( "//*[@id='product-section']/div[2]/div[2]/a/img").click()
time.sleep(2)
schiaparelli_open_click = browser.find_by_xpath( "//*[@id='wide-image-toggle']").click()
time.sleep(1)
schiaparelli_image = browser.html
soup = bs(schiaparelli_image, "html.parser")
schiaparelli_url = soup.find("img", class_="wide-image")["src"]
schiaparelli_img_url = hemisphere_base_url + schiaparelli_url
print(schiaparelli_img_url)
schiaparelli_title = soup.find("h2",class_="title").text
print(schiaparelli_title)
back_button = browser.find_by_xpath("//*[@id='splashy']/div[1]/div[1]/div[3]/section/a").click()
schiaparelli = {"image title":schiaparelli_title, "image url": schiaparelli_img_url}
hemisphere_img_urls.append(schiaparelli)

In [13]:

results1 = browser.find_by_xpath( "//*[@id='product-section']/div[2]/div[3]/a/img").click()
time.sleep(2)
syrtis_major_open_click = browser.find_by_xpath( "//*[@id='wide-image-toggle']").click()
time.sleep(1)
syrtis_major_image = browser.html
soup = bs(syrtis_major_image, "html.parser")
syrtis_major_url = soup.find("img", class_="wide-image")["src"]
syrtis_major_img_url = hemisphere_base_url + syrtis_major_url
print(syrtis_major_img_url)
syrtis_major_title = soup.find("h2",class_="title").text
print(syrtis_major_title)
back_button = browser.find_by_xpath("//*[@id='splashy']/div[1]/div[1]/div[3]/section/a").click()
syrtis_major = {"image title":syrtis_major_title, "image url": syrtis_major_img_url}
hemisphere_img_urls.append(syrtis_major)

AttributeError: 'ElementList' object has no attribute 'click'

In [24]:

results1 = browser.find_by_xpath( "//*[@id='product-section']/div[2]/div[4]/a/img").click()
time.sleep(2)
valles_marineris_open_click = browser.find_by_xpath( "//*[@id='wide-image-toggle']").click()
time.sleep(1)
valles_marineris_image = browser.html
soup = bs(valles_marineris_image, "html.parser")
valles_marineris_url = soup.find("img", class_="wide-image")["src"]
valles_marineris_img_url = hemisphere_base_url + syrtis_major_url
print(valles_marineris_img_url)
valles_marineris_title = soup.find("h2",class_="title").text
print(valles_marineris_title)
back_button = browser.find_by_xpath("//*[@id='splashy']/div[1]/div[1]/div[3]/section/a").click()
valles_marineris = {"image title":valles_marineris_title, "image url": valles_marineris_img_url}
hemisphere_img_urls.append(valles_marineris)

https://astrogeology.usgs.gov//cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg
Valles Marineris Hemisphere Enhanced


In [14]:
hemisphere_img_urls

NameError: name 'hemisphere_img_urls' is not defined

TypeError: document must be an instance of dict, bson.son.SON, bson.raw_bson.RawBSONDocument, or a type that inherits from collections.MutableMapping